In [1]:
!pip install --upgrade tensorflow --user
!pip install tf-quant-finance
!pip install QuantLib-Python

     |████████████████████████████████| 462 kB 4.3 MB/s 
     |████████████████████████████████| 1.4 MB 4.5 MB/s 
     |████████████████████████████████| 17.8 MB 180 kB/s 


In [2]:
# Get GPU info
!nvidia-smi

Mon Apr 25 17:18:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Get CPU info
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [4]:
from QuantLib import *
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tf_quant_finance as tff 
import tensorflow as tf
from scipy.interpolate import interp1d
import time
import pandas as pd

# Shortcut alias
pde = tff.math.pde
option_price = tff.black_scholes.option_price
implied_vol = tff.black_scholes.implied_vol

In [5]:
# tf.function decorator makes the function faster in graph mode.
@tf.function
def american_option(number_grid_points,
                    time_delta,
                    strike,
                    volatility,
                    risk_free_rate,
                    dividend_rates,
                    expiry,
                    spot,
                    european_exercise=False,
                    nb_bound=3.0,
                    dtype=tf.float64):
  """ Computes American Call options prices.

  Args:
    number_grid_points: A Python int. Number of grid points for the finite
      difference scheme.
    time_delta: A Python float. Grid time discretization parameter.
    strike: A real `Tensor` of shape `(number_of_options, 1)`.
      Represents the strikes of the underlying American options. 
    volatility: A real `Tensor` of shape `(number_of_options, 1)`.
      Represents the volatilities of the underlying American options. 
    risk_free_rate: A real `Tensor` of shape `(number_of_options, 1)`.
      Represents the risk-free interest rates associated with the underlying
      American options.
    expiry: A Python float. Expiry date of the options. If the options
      have different expiries, volatility term has to adjusted to
      make expiries the same.
    dtype: Optional `tf.dtype` used to assert dtype of the input `Tensor`s.

  Returns:
    A tuple of the estimated option prices of shape
    `(number_of_options, number_grid_points)` and the corresponding `Tensor` 
    of grid locations of shape `(number_grid_points,)`.
  """
  # Define the coordinate grid
  s_min = 0.01 #spot/nb_bound #0.01
  s_max = 300. #nb_bound*spot #300.

  grid = pde.grids.uniform_grid(minimums=[s_min],
                                maximums=[s_max],
                                sizes=[number_grid_points],
                                dtype=dtype)

  # Define the values grid for the final condition
  s = grid[0]
  final_values_grid = tf.nn.relu(s - strike)

  # Define the PDE coefficient functions
  def second_order_coeff_fn(t, grid):
    del t
    s = grid[0]
    return [[volatility ** 2 * s ** 2 / 2]]

  def first_order_coeff_fn(t, grid):
    del t
    s = grid[0]
    return [(risk_free_rate-dividend_rates) * s]

  def zeroth_order_coeff_fn(t, grid):
    del t, grid
    return -risk_free_rate

  # Define the boundary conditions
  @pde.boundary_conditions.dirichlet
  def lower_boundary_fn(t, grid):
    del t, grid
    return tf.constant(0.0, dtype=dtype)

  @pde.boundary_conditions.dirichlet
  def upper_boundary_fn(t, grid):
    del grid
    return tf.squeeze(s_max - strike * tf.exp(-(risk_free_rate-dividend_rates) * (expiry - t)))

  # In order to price American option one needs to set option values to 
  # V(x) := max(V(x), max(x - strike, 0)) after each iteration
  def values_transform_fn(t, grid, values):
    del t
    s = grid[0]
    if european_exercise==False:
        values_floor = tf.nn.relu(s - strike)
        return grid, tf.maximum(values, values_floor)
    else:
        return grid, values

  # Solve
  estimate_values, estimate_grid, _, _ = \
    pde.fd_solvers.solve_backward(
      start_time=expiry,
      end_time=0,
      values_transform_fn=values_transform_fn,
      coord_grid=grid,
      values_grid=final_values_grid,
      time_step=time_delta,
      boundary_conditions=[(lower_boundary_fn, upper_boundary_fn)],
      second_order_coeff_fn=second_order_coeff_fn,
      first_order_coeff_fn=first_order_coeff_fn,
      zeroth_order_coeff_fn=zeroth_order_coeff_fn,
      dtype=dtype
    )
  return estimate_values, estimate_grid[0]

def tqf_price(number_grid_points,
                    time_delta,
                    strike,
                    volatility,
                    risk_free_rate,
                    dividend_rates,
                    expiry,
                    spot,
                    european_exercise=False,
                    nb_bound=3.0,
                    dtype=tf.float64):
    estimate, grid_locations = american_option(
    time_delta=time_delta,
    expiry=tff.datetime.daycount_actual_actual_isda(
          start_date=tff.datetime.dates_from_tuples([(1998, 5, 15)]), 
          end_date=tff.datetime.dates_from_tuples([(1999, 5, 15)]),
          dtype=tf.float64).numpy()[0],
    number_grid_points=number_grid_points,
    volatility=tf.constant(0.20,tf.float64),
    risk_free_rate=tf.constant(0.02,tf.float64),
    dividend_rates=tf.constant(0.07,tf.float64),
    strike=tf.constant(40.0,tf.float64),
    spot=36.0,
    european_exercise=True,
    nb_bound=3.0,
    dtype=tf.float64)

    # Convert to numpy for plotting
    estimate = estimate.numpy()
    grid_locations = grid_locations.numpy()
    tqf_euro_price_pde = interp1d(grid_locations, estimate)(36.0)
    return tqf_euro_price_pde

In [6]:
device = "/cpu:0"
with tf.device(device):

    time_delta = 0.005
    number_grid_points = 1024

    batch_of_options = [5, 10, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000, 10000]
    for number_of_options in batch_of_options:
        filename = "https://raw.githubusercontent.com/arthurpham/google_colab/main/data/American_Option_Black_Scholes_GenerateCSV_{}.csv".format(number_of_options)
        option_inputs_df = pd.read_csv(filename)
        
        volatilities = option_inputs_df['volatility']
        risk_free_rates = option_inputs_df['risk_free_rate']
        strikes = option_inputs_df['strike']
        expiries = option_inputs_df['expiry']

        estimate, grid_locations = american_option(
            time_delta=time_delta,
            expiry=tff.datetime.daycount_actual_actual_isda(
                start_date=tff.datetime.dates_from_tuples([(1998, 5, 15)]), 
                end_date=tff.datetime.dates_from_tuples([(1999, 5, 15)]),
                dtype=tf.float64).numpy()[0],
            number_grid_points=number_grid_points,
            volatility=tf.constant(volatilities[0], dtype=tf.float64, shape=[len(strikes), 1]  ),
            risk_free_rate=tf.constant(risk_free_rates[0], dtype=tf.float64, shape=[len(strikes), 1]  ),
            dividend_rates=tf.constant(np.random.rand(number_of_options, 1) *0.0, tf.float64 ),
            strike=tf.constant(strikes, dtype=tf.float64, shape=[len(strikes), 1] ),
            spot=36.0,
            european_exercise=False,
            nb_bound=3.0,
            dtype=tf.float64)

        t = time.time()
    
        estimate, grid_locations = american_option(
            time_delta=time_delta,
            expiry=tff.datetime.daycount_actual_actual_isda(
                start_date=tff.datetime.dates_from_tuples([(1998, 5, 15)]), 
                end_date=tff.datetime.dates_from_tuples([(1999, 5, 15)]),
                dtype=tf.float64).numpy()[0],
            number_grid_points=number_grid_points,
            volatility=tf.constant(volatilities[0], dtype=tf.float64, shape=[len(strikes), 1]  ),
            risk_free_rate=tf.constant(risk_free_rates[0], dtype=tf.float64, shape=[len(strikes), 1]  ),
            dividend_rates=tf.constant(np.random.rand(number_of_options, 1) *0.0, tf.float64 ),
            strike=tf.constant(strikes, dtype=tf.float64, shape=[len(strikes), 1] ),
            spot=36.0,
            european_exercise=False,
            nb_bound=3.0,
            dtype=tf.float64)

        tqf_price_pde = []
        # print(estimate.numpy())
        for row_estimate in estimate.numpy():
            cur_price = interp1d(grid_locations, row_estimate)(option_inputs_df['strike'][0])
            tqf_price_pde.append(cur_price.item())
        time_tqf = time.time() - t

        tqf_options_per_second = number_of_options / time_tqf
        print('------------------------')
        print('TQF CPU {} options'.format(number_of_options))
        print('wall time: ', time_tqf)
        print('options per second: ', tqf_options_per_second)
        print('------------------------')
        print(tqf_price_pde[:10])

------------------------
TQF CPU 5 options
wall time:  0.2425222396850586
options per second:  20.61666594574189
------------------------
[8.741421742482567, 14.122058548118662, 0.11543072878025788, 0.6276506809102737, 0.31767447221991757]
------------------------
TQF CPU 10 options
wall time:  0.3089869022369385
options per second:  32.36383137150507
------------------------
[10.905464126104723, 0.3751709360584883, 0.5880509842870334, 5.318434679779265, 5.9621987213877565, 5.926924452221554, 3.777450187555818, 1.6949697843950968, 2.3655706630667557, 3.964053581338498]
------------------------
TQF CPU 100 options
wall time:  2.2047832012176514
options per second:  45.35593338373237
------------------------
[16.45277735124444, 5.2155976405006435, 9.299773826584689, 6.509145770536692, 3.3553537895830483, 10.544041004588191, 7.7705750957255235, 4.277853364639742, 10.97681333249017, 14.943005276375692]
------------------------
TQF CPU 150 options
wall time:  3.5077602863311768
options per 

In [7]:
device = "/gpu:0"
with tf.device(device):

    time_delta = 0.005
    number_grid_points = 1024

    batch_of_options = [5, 10, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000, 10000]
    for number_of_options in batch_of_options:
        filename = "https://raw.githubusercontent.com/arthurpham/google_colab/main/data/American_Option_Black_Scholes_GenerateCSV_{}.csv".format(number_of_options)
        option_inputs_df = pd.read_csv(filename)
        
        volatilities = option_inputs_df['volatility']
        risk_free_rates = option_inputs_df['risk_free_rate']
        strikes = option_inputs_df['strike']
        expiries = option_inputs_df['expiry']

        estimate, grid_locations = american_option(
            time_delta=time_delta,
            expiry=tff.datetime.daycount_actual_actual_isda(
                start_date=tff.datetime.dates_from_tuples([(1998, 5, 15)]), 
                end_date=tff.datetime.dates_from_tuples([(1999, 5, 15)]),
                dtype=tf.float64).numpy()[0],
            number_grid_points=number_grid_points,
            volatility=tf.constant(volatilities[0], dtype=tf.float64, shape=[len(strikes), 1]  ),
            risk_free_rate=tf.constant(risk_free_rates[0], dtype=tf.float64, shape=[len(strikes), 1]  ),
            dividend_rates=tf.constant(np.random.rand(number_of_options, 1) *0.0, tf.float64 ),
            strike=tf.constant(strikes, dtype=tf.float64, shape=[len(strikes), 1] ),
            spot=36.0,
            european_exercise=False,
            nb_bound=3.0,
            dtype=tf.float64)

        t = time.time()
    
        estimate, grid_locations = american_option(
            time_delta=time_delta,
            expiry=tff.datetime.daycount_actual_actual_isda(
                start_date=tff.datetime.dates_from_tuples([(1998, 5, 15)]), 
                end_date=tff.datetime.dates_from_tuples([(1999, 5, 15)]),
                dtype=tf.float64).numpy()[0],
            number_grid_points=number_grid_points,
            volatility=tf.constant(volatilities[0], dtype=tf.float64, shape=[len(strikes), 1]  ),
            risk_free_rate=tf.constant(risk_free_rates[0], dtype=tf.float64, shape=[len(strikes), 1]  ),
            dividend_rates=tf.constant(np.random.rand(number_of_options, 1) *0.0, tf.float64 ),
            strike=tf.constant(strikes, dtype=tf.float64, shape=[len(strikes), 1] ),
            spot=36.0,
            european_exercise=False,
            nb_bound=3.0,
            dtype=tf.float64)

        tqf_price_pde = []
        # print(estimate.numpy())
        for row_estimate in estimate.numpy():
            cur_price = interp1d(grid_locations, row_estimate)(option_inputs_df['strike'][0])
            tqf_price_pde.append(cur_price.item())
        time_tqf = time.time() - t

        tqf_options_per_second = number_of_options / time_tqf
        print('------------------------')
        print('TQF GPU {} options'.format(number_of_options))
        print('wall time: ', time_tqf)
        print('options per second: ', tqf_options_per_second)
        print('------------------------')
        print(tqf_price_pde[:10])

------------------------
TQF GPU 5 options
wall time:  0.3330082893371582
options per second:  15.01464125698592
------------------------
[8.741421742482634, 14.122058548118725, 0.11543072878025808, 0.6276506809102814, 0.31767447221992035]
------------------------
TQF GPU 10 options
wall time:  0.32201218605041504
options per second:  31.05472535885451
------------------------
[10.905464126103896, 0.3751709360584539, 0.588050984286965, 5.318434679778833, 5.962198721387219, 5.926924452221086, 3.777450187555436, 1.6949697843949332, 2.3655706630665296, 3.964053581338143]
------------------------
TQF GPU 100 options
wall time:  0.4231867790222168
options per second:  236.30227823055435
------------------------
[16.45277735124422, 5.215597640500625, 9.299773826584481, 6.509145770536785, 3.3553537895829857, 10.544041004588149, 7.770575095725207, 4.277853364639866, 10.976813332490144, 14.94300527637581]
------------------------
TQF GPU 150 options
wall time:  0.4854395389556885
options per se